### Adjust Word Group Repeat With Youtube Link

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 20000  # 28 native word end index

# file extention
file_ext = "Twogram"

In [5]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 5/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [7]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Result/2-Find Word Group Youtube Link"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [8]:
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,lQ6ziU9vAzY,67,69,hayır hayır
1,5mHjvWNNC_Y,478,480,hayır hayır
2,Yr2ZLfWT0i8,5615,5617,bu bir
3,DAh5wI1W3qw,706,708,bu bir
4,_s9OA69wZ2A,377,379,bu bir
...,...,...,...,...
634,4m3OoIpXGwc,492,494,herhangi bir
635,GwbonuGbL9k,1259,1260,de bu
636,rd_spD2gMdE,188,190,de bu
637,OEhdJ0sK27I,42,44,de bu


In [9]:
df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
df_link_selected_manuel

,video_id,start_time,end_time,text,video_url
0,lQ6ziU9vAzY,67,69,hayır hayır,https://www.youtube.com/watch?v=lQ6ziU9vAzY&t=67s
1,5mHjvWNNC_Y,478,480,hayır hayır,https://www.youtube.com/watch?v=5mHjvWNNC_Y&t=...
2,Yr2ZLfWT0i8,5615,5617,bu bir,https://www.youtube.com/watch?v=Yr2ZLfWT0i8&t=...
3,DAh5wI1W3qw,706,708,bu bir,https://www.youtube.com/watch?v=DAh5wI1W3qw&t=...
4,_s9OA69wZ2A,377,379,bu bir,https://www.youtube.com/watch?v=_s9OA69wZ2A&t=...
...,...,...,...,...,...
634,4m3OoIpXGwc,492,494,herhangi bir,https://www.youtube.com/watch?v=4m3OoIpXGwc&t=...
635,GwbonuGbL9k,1259,1260,de bu,https://www.youtube.com/watch?v=GwbonuGbL9k&t=...
636,rd_spD2gMdE,188,190,de bu,https://www.youtube.com/watch?v=rd_spD2gMdE&t=...
637,OEhdJ0sK27I,42,44,de bu,https://www.youtube.com/watch?v=OEhdJ0sK27I&t=42s


In [10]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,bu şekilde,9
1,daha sonra,9
2,ve bu,9
3,bir yer,9
4,ve bir,9
...,...,...
168,de öyle,1
169,senin gibi,1
170,ciddi misin,1
171,bırak beni,1


In [11]:
df_count = df_count[df_count["count"] > 2]

In [12]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,video_url,count
0,1LSDvAXn2Ug,7184,7186,ve bu,https://www.youtube.com/watch?v=1LSDvAXn2Ug&t=...,9
1,eJrrDQe1scs,2110,2111,ve bu,https://www.youtube.com/watch?v=eJrrDQe1scs&t=...,9
2,zzXTIIo7EvM,923,925,ve bu,https://www.youtube.com/watch?v=zzXTIIo7EvM&t=...,9
3,30Sd_IpkgK4,460,461,ve bu,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,9
4,5Dxtviu5tuU,394,396,ve bu,https://www.youtube.com/watch?v=5Dxtviu5tuU&t=...,9
...,...,...,...,...,...,...
542,bt7g4VoVBoE,3698,3700,alabilir miyim,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...,3
543,3ogB_jaW4FI,3771,3773,alabilir miyim,https://www.youtube.com/watch?v=3ogB_jaW4FI&t=...,3
544,VmMoYRKRRso,400,402,adın ne,https://www.youtube.com/watch?v=VmMoYRKRRso&t=...,3
545,3PDJDym1ZXQ,2321,2323,adın ne,https://www.youtube.com/watch?v=3PDJDym1ZXQ&t=...,3


In [100]:
df_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 {file_ext} Video List.xlsx", index=False)

In [101]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx", index=False)

##### Link Repeat

In [102]:
path_folder_file2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Result/2-Find Word Group Youtube Link/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Selected_Count_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [103]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text,video_url,count
0,ALQGHRcrASE,635,637,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=ALQGHRcrASE&t=...,6
1,Jl9__Gs09BI,2708,2710,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=Jl9__Gs09BI&t=...,6
2,f694PFHVMFM,6698,6700,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=f694PFHVMFM&t=...,6
3,fhG1ArBe_uU,2719,2722,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=fhG1ArBe_uU&t=...,6
4,3PDJDym1ZXQ,4593,4596,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=3PDJDym1ZXQ&t=...,6
5,UrEJ7oAhcX8,5106,5109,söylemek istediğiniz bir şey var mı,https://www.youtube.com/watch?v=UrEJ7oAhcX8&t=...,6
6,hM5s4liNn7g,830,833,istediğin başka bir şey var mı,https://www.youtube.com/watch?v=hM5s4liNn7g&t=...,6
7,vgHcue8Uu9I,813,816,istediğin başka bir şey var mı,https://www.youtube.com/watch?v=vgHcue8Uu9I&t=...,6
8,kQkG6D4c_xU,1077,1079,istediğin başka bir şey var mı,https://www.youtube.com/watch?v=kQkG6D4c_xU&t=...,6
9,ao2kpL29HwI,454,457,istediğin başka bir şey var mı,https://www.youtube.com/watch?v=ao2kpL29HwI&t=...,6


In [104]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

5

In [105]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,text,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [106]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [107]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [108]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"ALQGHRcrASE,Jl9__Gs09BI,f694PFHVMFM,fhG1ArBe_u...","635,2708,6698,2719,4593,5106,635,5,635,5,635,5","637,2710,6700,2722,4596,5109,637,8,637,8,637,8",söylemek istediğiniz bir şey var mı,6.0,"[play_lms_videos video_id=ALQGHRcrASE,Jl9__Gs0..."
1,"hM5s4liNn7g,vgHcue8Uu9I,kQkG6D4c_xU,ao2kpL29Hw...","830,813,1077,454,815,649,830,5,830,5,830,5","833,816,1079,457,818,651,833,8,833,8,833,8",istediğin başka bir şey var mı,6.0,"[play_lms_videos video_id=hM5s4liNn7g,vgHcue8U..."
2,"9WCC0iXY4Tk,TVcNvuRpBYM,jGMsIgTA_wk,9ISf_UoeXU...","905,304,2075,577,105,905,5,905,5,905,5","907,306,2078,580,108,907,8,907,8,907,8",o kadar çok şey var ki,5.0,"[play_lms_videos video_id=9WCC0iXY4Tk,TVcNvuRp..."
3,"e0ldv6mrDn8,tCbQEuQKW3A,fwQJha07Gj4,e0ldv6mrDn...","919,3702,817,919,5,919,5,919,5","923,3704,820,923,10,923,10,923,10",söylemek istediğin bir şey var mı,3.0,"[play_lms_videos video_id=e0ldv6mrDn8,tCbQEuQK..."
4,"VxWNvMpYujU,VxWNvMpYujU,EtrRFFyvFKI,VxWNvMpYuj...","1684,2498,7,2498,5,2498,5,2498,5","1687,2502,10,2502,10,2502,10,2502,10",gereken başka bir şey var mı,3.0,"[play_lms_videos video_id=VxWNvMpYujU,VxWNvMpY..."


#### Copy Move And Delete

In [109]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_10000_Word_Youtube_Talk_Time_Join_Sixgram.xlsx']

In [110]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [111]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass